In [2]:
import pandas as pd
import numpy as np

import re

In [3]:
train = pd.read_csv(r'C:\Users\Admin\OneDrive\Desktop\student_resource\dataset\train.csv')
test = pd.read_csv(r'C:\Users\Admin\OneDrive\Desktop\student_resource\dataset\test.csv')

In [4]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sample_id        75000 non-null  int64  
 1   catalog_content  75000 non-null  object 
 2   image_link       75000 non-null  object 
 3   price            75000 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.3+ MB
None


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_percentage_error

In [10]:
import torch
from transformers import AutoTokenizer

In [11]:
import unicodedata

def clean_text(text):
    text = unicodedata.normalize('NFKD', text)
    text = text.replace('â€™', "'").replace('â€œ', '"').replace('â€', '"')
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s\.]', '', text)
    
    return text.strip()
train['clean_text'] = train['catalog_content'].apply(clean_text)


In [12]:
def extract_value_unit(text):
    value_match = re.search(r'Value:\s*([\d\.]+)',text)
    unit_match = re.search(r'Unit:\s*(\w+)',text)
    value = float(value_match.group(1)) if value_match else None
    unit = unit_match.group(1).lower() if unit_match else None
    return value, unit 


In [13]:
train[['value','unit']] = train['catalog_content'].apply(lambda x: pd.Series(extract_value_unit(x)))

In [14]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
training_encodings = tokenizer(
    train['clean_text'].tolist(),
    truncation=True,
    padding=True,
    max_length=128
)

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
